# Linear transformations over data matrix

---
> Erick Eduardo Aguilar Hernández:
> * mat.ErickAguilar@gmail.com.mx
> * isc.ErickAguilar@gmail.com

---

Let $V$, $W$ be a vector spaces over $\mathbb{R}$  with $dim(V)=p$, $dim(W)=m$. Let $T: V \subset \mathbb{R}^p$ $\rightarrow$ $W \subset \mathbb{R}^m$ be a linear transformation. Then there exists a unique $p \times m$ matrix $\mathbf{A}$ such that $T\left[v\right]=A\left[v\right]$ for every $v \in V$. 

In some types of analysis a linear transformation of the data matrix it's required, so that each observation should be transformed to another vector space, note that each row could be interpreted as a vector in $\mathbb{R}^p$, so the matrix expression associated to the linear transformation have be expressed as:

$$
\begin{align*} 
\mathbf{Y}_{m\times n}  & = \mathbf{A}_{m\times p}[\mathbf{X}]'_{p\times n}  \\
\mathbf{Y'}_{n\times m} & = \mathbf{X}_{n\times p}[\mathbf{A}]'_{p\times m} \\
\mathbf{Z}_{n\times m}  & = \mathbf{X}_{n\times p}[\mathbf{B}]_{p\times m} \\
\end{align*}
$$

This transformation produces a new data matrix $\textbf{Z}$.

$$
\left[ \begin{array}{ccccc}
z_{1 1} & \cdots & z_{1 m} \\
\vdots  & \ddots & \vdots \\
z_{i 1} & \cdots & z_{i m} \\
\vdots  & \ddots & \vdots \\
z_{n 1} & \cdots & z_{n m} \end{array} \right]
= \left[ \begin{array}{ccccc}
x_{1 1} & \cdots & x_{1 j} & \cdots & x_{1 p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{i 1} & \cdots & x_{i j} & \cdots & x_{i p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{n 1} & \cdots & x_{n j} & \cdots & x_{n p} \end{array} \right]
\left[ \begin{array}{ccccc}
b_{1 1} & \cdots & b_{1 m} \\
\vdots  & \ddots & \vdots \\
b_{j 1} & \cdots & b_{i m} \\
\vdots  & \ddots & \vdots \\
b_{p 1} & \cdots & b_{p m} \end{array} \right]
$$

Where the k-th column could be expressed in terms of linear combinations:

$$
\begin{align*}
z_{1 k}   & = b_{1 k} x_{1 1} + \cdots b_{j k} x_{1 j} + \cdots + b_{p k} x_{1 p}   \\
\vdots    & = \vdots \\
z_{n k}   & = b_{1 k} x_{1 1} + \cdots b_{j k} x_{1 j} + \cdots + b_{p k} x_{1 p} \\
\end{align*}
$$

Properties:
*  $\mathbf{\bar{y}}=\frac{1}{n}[\textbf{1}_{1\times n}]\textbf{Y} = \frac{1}{n}[\textbf{1}_{1\times n}]\textbf{XA} = \mathbf{\bar{x}A}$
   
*  $\mathbf{S_y}=\frac{1}{n}\textbf{Y'HY}=\frac{1}{n}\textbf{[XA]'H[XA]}=\frac{1}{n}\textbf{A'X'HXA}=\textbf{A'}\frac{1}{n}\textbf{X'HX}\textbf{A}=\textbf{A'SA}$

Consider the centered data matrix $\textbf{Y} = \textbf{HX}$

#### Proyection matrix
---

If $V$ it's a subspace of $\mathbb{R}^p$ whose basis is $\{\mathbf{\alpha}_1,\dotsc,\mathbf{\alpha}_k\}$ then $\forall \mathbf{v} \in V$ with $x_i=1,\dotsc, k \in \mathbb{R}$


$$
\begin{align*} 
\mathbf{v} & = x_1 \mathbf{\alpha}_1+\dotsc+x_k \mathbf{\alpha}_k \\
 & = x_1
\left[ \begin{array}{c}
\alpha_{1 1}\\
\vdots \\
\alpha_{p 1}\end{array} \right]
+\dotsc+x_k
\left[ \begin{array}{c}
\alpha_{1 k}\\
\vdots \\
\alpha_{p k}\end{array} \right] \\
& = \left[ \begin{array}{ccccc}
x_1 \alpha_{1 1} + & \cdots & + x_k \alpha_{1 k} \\
\vdots  & \ddots & \vdots  \\
x_1 \alpha_{p 1} + & \cdots & + x_k \alpha_{p k} \end{array} \right]\\
& = \left[ \begin{array}{ccccc}
\alpha_{1 1} & \cdots & \alpha_{1 k} \\
\vdots  & \ddots & \vdots  \\
\alpha_{p 1} & \cdots & \alpha_{p k} \end{array} \right]
\left[ \begin{array}{ccccc}
x_1 \\
\vdots \\
x_k \end{array} \right]\\
\mathbf{v} &= \mathbf{A}_{p \times k} \mathbf{x}_{k \times 1}
\end{align*}
$$

if $\textbf{y} \in \mathbb{R}^p $ then $Proy_V\textbf{y} \in V$ then $Proy_V\textbf{y}=\mathbf{A} \mathbf{x}$ for some $\textbf{x} \in \mathbb{R}^k $ also $\textbf{y} = Proy_V\textbf{y}+Proy_{V^{\perp}}\textbf{y}$ where $V^{\perp} = \left< \{\mathbf{\alpha}_1,\dotsc,\mathbf{\alpha}_k\}\right>^{\perp}=Kernel(A')$ then

$$
\begin{align*}
Proy_{V^{\perp}}\textbf{y} & = \textbf{y} - Proy_V\textbf{y} \in Kernel(A')\\
\implies \textbf{A'}\left(\textbf{y} - Proy_V\textbf{y}\right) & = \textbf{0}\\
\textbf{A'}\textbf{y} - \textbf{A'} Proy_V\textbf{y} & = \textbf{0}\\
\textbf{A'}\textbf{y} - \textbf{A'} \mathbf{A}\mathbf{x} & = \textbf{0}\\
\textbf{A'}\textbf{y} & = \textbf{A'}\mathbf{A}\mathbf{x} \\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\mathbf{A}\mathbf{x} \\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{A}^{-1}[\textbf{A'}]^{-1}\textbf{A'}\mathbf{A}\mathbf{x} \\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{I}\mathbf{x}\\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{x} \\
\mathbf{A}\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{Ax}\\
\mathbf{A}\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = Proy_V\textbf{y}\\
\end{align*}
$$

**Observation:**

The centering matrix $\textbf{H}$ it's a projection matrix of the rows $\textbf{x}_i$ of the data matrix, onto the $(n−1)$ dimensional subspace that is orthogonal to the nullspace $\mathbf{1}$ (This is the subspace of all n-vectors whose components sum to zero).